In [11]:
import math
from collections import Counter, defaultdict
from functools import partial

In [12]:
def entropy(class_probabilities):
    return (sum(-p * math.log(p,2)
               for p in class_probabilities
               if p != 0))

def class_probabilities(labels):
    total_count = len(labels)
    return [count / total_count
           for count in Counter(labels).values()]

def data_entropy(labeled_data):
    labels = [label for _, label in labeled_data]
    probabilities = class_probabilities(labels)
    return entropy(probabilities)

In [13]:
def partition_entropy(subsets):
    total_count = sum(len(subset)
                     for subset in subsets)
    return sum(data_entropy(subset) * len(subset) / total_count
              for subset in subsets)

In [14]:
inputs = [
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'no'},   False),
        ({'level':'Senior','lang':'Java','tweets':'no','phd':'yes'},  False),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'no'},     True),
        ({'level':'Junior','lang':'Python1', 'tweets':'no','phd':'no'},  True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'R','tweets':'yes','phd':'yes'},    False),
        ({'level':'Mid','lang':'R','tweets':'yes','phd':'yes'},        True),
        ({'level':'Senior','lang':'Python','tweets':'no','phd':'no'}, False),
        ({'level':'Senior','lang':'R','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'yes','phd':'no'}, True),
        ({'level':'Senior','lang':'Python','tweets':'yes','phd':'yes'},True),
        ({'level':'Mid','lang':'Python','tweets':'no','phd':'yes'},    True),
        ({'level':'Mid','lang':'Java','tweets':'yes','phd':'no'},      True),
        ({'level':'Junior','lang':'Python','tweets':'no','phd':'yes'},False)
    ]

In [15]:
def partition_by(inputs, attribute):
    groups = defaultdict(list)
    
    for input in inputs:
        key = input[0][attribute]
        groups[key].append(input)
    return groups

In [16]:
partition_by(inputs, 'lang')

defaultdict(list,
            {'Java': [({'level': 'Senior',
                'lang': 'Java',
                'tweets': 'no',
                'phd': 'no'},
               False),
              ({'level': 'Senior',
                'lang': 'Java',
                'tweets': 'no',
                'phd': 'yes'},
               False),
              ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'},
               True)],
             'Python': [({'level': 'Mid',
                'lang': 'Python',
                'tweets': 'no',
                'phd': 'no'},
               True),
              ({'level': 'Senior',
                'lang': 'Python',
                'tweets': 'no',
                'phd': 'no'},
               False),
              ({'level': 'Junior',
                'lang': 'Python',
                'tweets': 'yes',
                'phd': 'no'},
               True),
              ({'level': 'Senior',
                'lang': 'Python',
                'tweets': 'yes

In [17]:
def partition_entropy_by(inputs, attribute):
    partitions = partition_by(inputs, attribute)
    return partition_entropy(partitions.values())

In [18]:
partition_by(inputs, 'lang').items()

dict_items([('Java', [({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False), ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True)]), ('Python', [({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False), ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True), ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True), ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, False)]), ('Python1', [({'level': 'Junior', 'lang': 'Python1', 'tweets': 'no', 'phd': 'no'}, True)]), ('R', [({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True), ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False), ({'level': 'Mid', 'lang': 'R', 'tweet

In [19]:
inputs[0][0].keys()

dict_keys(['level', 'lang', 'tweets', 'phd'])

In [20]:
for key in ['level', 'lang', 'tweets', 'phd']:
    print (key, partition_entropy_by(inputs, key))

level 0.6935361388961919
lang 0.8221267860233527
tweets 0.7884504573082896
phd 0.8921589282623617


In [21]:
senior_inputs = [(input, label)
                for input, label in inputs if input['level'] == 'Senior']

In [22]:
for key in ['lang', 'tweets', 'phd']:
    print (key, partition_entropy_by(senior_inputs, key))

lang 0.4
tweets 0.0
phd 0.9509775004326938


In [23]:
def classify(tree, input):
    if tree in [True, False]:
        return tree

In [24]:
def build_tree(inputs, split_candidates = None):
    if split_candidates is None:
        split_candidates = inputs[0][0].keys()
    
    num_inputs = len(inputs)
    num_trues = len([label for item, label in inputs if label == True])
    num_falses = num_inputs - num_trues
    
    if num_trues == 0: return False
    if num_falses == 0: return True
    
    if not split_candidates:
        return num_trues >= num_fales
    
    find_best_split = min(split_candidates,
                         key = partial(partition_entropy_by, inputs))
    
    partitions = partition_by(inputs, find_best_split)
    
    new_candidates = [i for i in split_candidates]
    
    subtrees = { attribute_value: build_tree(subset, new_candidates)
               for attribute_value, subset in partitions.items()}
    
    subtrees[None] = num_trues > num_falses
    
    return (find_best_split, subtrees)

In [25]:
def classify(tree, input):
    """classify the input using the given decision tree"""
    
    # if this is a leaf node, return its value
    if tree in [True, False]:
        return tree
   
    # otherwise find the correct subtree
    attribute, subtree_dict = tree
    
    subtree_key = input.get(attribute)  # None if input is missing attribute

    if subtree_key not in subtree_dict: # if no subtree for key,
        subtree_key = None              # we'll use the None subtree
    
    subtree = subtree_dict[subtree_key] # choose the appropriate subtree
    return classify(subtree, input)     # and use it to classify the input
    

In [29]:
build_tree(inputs)

('level',
 {'Senior': ('tweets', {'no': False, 'yes': True, None: False}),
  'Mid': True,
  'Junior': ('phd', {'no': True, 'yes': False, None: True}),
  None: True})

In [28]:
classify(build_tree(inputs),{"level":'intern'
                            })

True